<a href="https://colab.research.google.com/github/jathurT/Data-Crunch-Competition/blob/main/%20Notebooks_and_Scripts/Sample%20V1.0.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis

In [537]:
# Importing Libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [538]:
import warnings
warnings.filterwarnings('ignore')

In [539]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [540]:
# Load data
train_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/DataCrunchCompetitionDatasets/sample_submission.csv')

In [541]:
# preview the dataset
train_df.head()

,ID,Year,Month,Day,kingdom,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
0,1,1,4,1,Arcadia,24.280002,-37.229980,25.50,30.50,8.5,10.3,22.52,58.89,16,8.6,283,1.648659
1,2,1,4,1,Atlantis,22.979999,-37.329990,299.65,305.15,5.9,8.2,22.73,11.83,12,15.8,161,1.583094
2,3,1,4,1,Avalon,22.880000,-37.130006,26.30,31.50,5.2,6.4,22.73,11.83,12,15.8,161,1.593309
3,4,1,4,1,Camelot,24.180003,-36.929994,24.00,28.40,8.2,10.7,22.67,75.27,16,6.4,346,1.638997
4,5,1,4,1,Dorne,25.780002,-37.530000,28.00,32.80,5.7,10.2,22.35,4.81,8,16.7,185,1.719189


*   Some temperature measurements may be in **Celsius or Kelvin** depending on the kingdom




In [542]:
# preview the dataset
test_df.head()

,ID,Year,Month,Day,kingdom
0,84961,9,1,1,Arcadia
1,84962,9,1,1,Atlantis
2,84963,9,1,1,Avalon
3,84964,9,1,1,Camelot
4,84965,9,1,1,Dorne


In [543]:
# preview the dataset
submission_df.head()

,ID,Avg_Temperature,Radiation,Rain_Amount,Wind_Speed,Wind_Direction
0,84961,0,0,0,0,0
1,84962,0,0,0,0,0
2,84963,0,0,0,0,0
3,84964,0,0,0,0,0
4,84965,0,0,0,0,0


In [544]:
train_df.shape

(84960, 17)

We can see that there are **84960 instances** and **17 variables** in the train data set.

In [545]:
test_df.shape

(4530, 5)

We can see that there are **4530 instances** and **5 variables** in the test data set.

In [546]:
submission_df.shape

(4530, 6)

We can see that there are **4530 instances** and **6 variables** in the data set.

In [547]:
train_col_names = train_df.columns
train_col_names

Index(['ID', 'Year', 'Month', 'Day', 'kingdom', 'latitude', 'longitude',
       'Avg_Temperature', 'Avg_Feels_Like_Temperature', 'Temperature_Range',
       'Feels_Like_Temperature_Range', 'Radiation', 'Rain_Amount',
       'Rain_Duration', 'Wind_Speed', 'Wind_Direction', 'Evapotranspiration'],
      dtype='object')

Here We can celarly see that some column names staring with uppercase letters. So, to standardize and maintain consistency in column naming conventions, we can change these uppercase columns to lowercase.

In [548]:
# train_df.columns = train_df.columns.str.lower()
# train_df.columns

In [549]:
# view summary of dataset
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84960 entries, 0 to 84959
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            84960 non-null  int64  
 1   Year                          84960 non-null  int64  
 2   Month                         84960 non-null  int64  
 3   Day                           84960 non-null  int64  
 4   kingdom                       84960 non-null  object 
 5   latitude                      84960 non-null  float64
 6   longitude                     84960 non-null  float64
 7   Avg_Temperature               84960 non-null  float64
 8   Avg_Feels_Like_Temperature    84960 non-null  float64
 9   Temperature_Range             84960 non-null  float64
 10  Feels_Like_Temperature_Range  84960 non-null  float64
 11  Radiation                     84960 non-null  float64
 12  Rain_Amount                   84960 non-null  float64
 13  R

**Types of variables**

---


In this section, We segregate the dataset into categorical and numerical variables. There are a mixture of categorical and numerical variables in the dataset. Categorical variables have data type object. Numerical variables have data type float64 and int64.

---



In [550]:
# find categorical variables
categorical = [var for var in train_df.columns if train_df[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)

There are 1 categorical variables

The categorical variables are : ['kingdom']


## Explore problems within categorical variables

In [551]:
# check missing values in categorical variables
train_df[categorical].isnull().sum()

,0
kingdom,0


In [552]:
# check for cardinality in categorical variables
for var in categorical:
    print(var, ' contains ', len(train_df[var].unique()), ' labels')

kingdom  contains  30  labels


High cardinality may pose some serious problems in the machine learning model. So, we will check for high cardinality.

In [553]:
train_df.kingdom.unique()

array(['Arcadia', 'Atlantis', 'Avalon', 'Camelot', 'Dorne', 'Eden',
       'El Dorado', 'Elysium', 'Emerald City', 'Helios', 'Krypton',
       'Metropolis', 'Midgar', 'Midgard', 'Mordor', 'Neo-City',
       'Neo-Tokyo', 'Nirvana', 'Olympus', 'Pandora', 'Rapture',
       'Rivendell', 'Serenity', 'Shangri-La', 'Solara', 'Solstice',
       'Sunspear', 'Utopia', 'Valyria', 'Winterfell'], dtype=object)

In [554]:
train_df.kingdom.value_counts()

,count
kingdom,
Arcadia,2832
Atlantis,2832
Avalon,2832
Camelot,2832
Dorne,2832
Eden,2832
El Dorado,2832
Elysium,2832
Emerald City,2832


We can clearly see that every label has same amount of tuples.
We want to do One Hot Encoding for kingdom variable in the data preprocessing step because of the 30 labels

## Explore Problem With Numerical Variables

In [555]:
# find numerical variables
numerical = [var for var in train_df.columns if train_df[var].dtype!='O']
print('There are {} numerical variables\n'.format(len(numerical)))
print('The numerical variables are :', numerical)

There are 16 numerical variables

The numerical variables are : ['ID', 'Year', 'Month', 'Day', 'latitude', 'longitude', 'Avg_Temperature', 'Avg_Feels_Like_Temperature', 'Temperature_Range', 'Feels_Like_Temperature_Range', 'Radiation', 'Rain_Amount', 'Rain_Duration', 'Wind_Speed', 'Wind_Direction', 'Evapotranspiration']


In [556]:
# view the numerical variables
train_df[numerical].head()

,ID,Year,Month,Day,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
0,1,1,4,1,24.280002,-37.229980,25.50,30.50,8.5,10.3,22.52,58.89,16,8.6,283,1.648659
1,2,1,4,1,22.979999,-37.329990,299.65,305.15,5.9,8.2,22.73,11.83,12,15.8,161,1.583094
2,3,1,4,1,22.880000,-37.130006,26.30,31.50,5.2,6.4,22.73,11.83,12,15.8,161,1.593309
3,4,1,4,1,24.180003,-36.929994,24.00,28.40,8.2,10.7,22.67,75.27,16,6.4,346,1.638997
4,5,1,4,1,25.780002,-37.530000,28.00,32.80,5.7,10.2,22.35,4.81,8,16.7,185,1.719189


In [557]:
# check missing values in numerical variables
train_df[numerical].isnull().sum()

,0
ID,0
Year,0
Month,0
Day,0
latitude,0
longitude,0
Avg_Temperature,0
Avg_Feels_Like_Temperature,0
Temperature_Range,0
Feels_Like_Temperature_Range,0


In [570]:
duplicates =train_df[train_df.duplicated()]
duplicates.shape

(0, 17)

**Summary of numerical variables**

---




*   There are 8 numerical variables.
*   All of the numerical variables are of continuous and discrete type.




In [558]:
target_cols = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']
print("\nBasic statistics of target variables:")
print(round(train_df[target_cols].describe()))


Basic statistics of target variables:
       Avg_Temperature  Radiation  Rain_Amount  Wind_Speed  Wind_Direction
count          84960.0    84960.0      84960.0     84960.0         84960.0
mean             136.0       20.0          8.0        16.0           216.0
std              134.0        4.0         13.0         6.0            94.0
min               19.0        3.0          0.0         2.0             0.0
25%               26.0       18.0          1.0        11.0           119.0
50%               28.0       21.0          3.0        15.0           255.0
75%              299.0       23.0          9.0        19.0           286.0
max              304.0       30.0        440.0        50.0           359.0


In [559]:
# Check what's actually in your data
print("Data types:")
print(train_df[['Year', 'Month', 'Day']].dtypes)

# Check value ranges
print("\nValue ranges:")
print("Year range:", train_df['Year'].min(), "to", train_df['Year'].max())
print("Month range:", train_df['Month'].min(), "to", train_df['Month'].max())
print("Day range:", train_df['Day'].min(), "to", train_df['Day'].max())

# Check for any unusual values
print("\nSample of first few rows:")
print(train_df[['Year', 'Month', 'Day']].head(10))

Data types:
Year     int64
Month    int64
Day      int64
dtype: object

Value ranges:
Year range: 1 to 8
Month range: 1 to 12
Day range: 1 to 31

Sample of first few rows:
   Year  Month  Day
0     1      4    1
1     1      4    1
2     1      4    1
3     1      4    1
4     1      4    1
5     1      4    1
6     1      4    1
7     1      4    1
8     1      4    1
9     1      4    1


In [560]:
train_df.head()

,ID,Year,Month,Day,kingdom,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
0,1,1,4,1,Arcadia,24.280002,-37.229980,25.50,30.50,8.5,10.3,22.52,58.89,16,8.6,283,1.648659
1,2,1,4,1,Atlantis,22.979999,-37.329990,299.65,305.15,5.9,8.2,22.73,11.83,12,15.8,161,1.583094
2,3,1,4,1,Avalon,22.880000,-37.130006,26.30,31.50,5.2,6.4,22.73,11.83,12,15.8,161,1.593309
3,4,1,4,1,Camelot,24.180003,-36.929994,24.00,28.40,8.2,10.7,22.67,75.27,16,6.4,346,1.638997
4,5,1,4,1,Dorne,25.780002,-37.530000,28.00,32.80,5.7,10.2,22.35,4.81,8,16.7,185,1.719189


In [561]:
# # Check time range
# print(f"\nTrain data time range: {train_df['date'].min()} to {train_df['date'].max()}")
# print(f"Test data time range: {test_df['date'].min()} to {test_df['date'].max()}")

In [562]:
 # Check kingdoms
print(f"\nNumber of kingdoms in train data: {train_df['kingdom'].nunique()}")
print(f"Number of kingdoms in test data: {test_df['kingdom'].nunique()}")


Number of kingdoms in train data: 30
Number of kingdoms in test data: 30


In [563]:
# Check temperature units
print("\nIdentifying temperature units by kingdom...")
kingdom_temp_stats = train_df.groupby('kingdom')['Avg_Temperature'].agg(['mean', 'min', 'max'])
kingdom_temp_stats['likely_unit'] = kingdom_temp_stats['mean'].apply(lambda x: 'Kelvin' if x > 100 else 'Celsius')
print(kingdom_temp_stats)


Identifying temperature units by kingdom...
                    mean     min     max likely_unit
kingdom                                             
Arcadia        25.644668   21.00   30.40     Celsius
Atlantis      300.362465  297.75  303.05      Kelvin
Avalon         26.603355   24.30   29.30     Celsius
Camelot        23.594456   18.60   27.90     Celsius
Dorne          27.552684   22.70   30.80     Celsius
Eden           26.772352   23.80   30.10     Celsius
El Dorado     298.369280  295.25  301.15      Kelvin
Elysium        26.672564   23.70   30.00     Celsius
Emerald City  299.786017  296.95  302.65      Kelvin
Helios         25.890501   21.00   30.50     Celsius
Krypton       299.972952  296.95  303.25      Kelvin
Metropolis     26.845480   23.80   29.90     Celsius
Midgar         27.173305   23.30   30.20     Celsius
Midgard        25.170621   19.90   29.10     Celsius
Mordor         27.414548   24.00   30.10     Celsius
Neo-City       26.676236   23.70   29.80     Celsius
N

# Data Preprocessing

In [564]:
print("\nSplitting data into train and validation sets...")
# Years 1-7 for training, Year 8 for validation
train_years = list(range(1, 8))
val_year = 8

train_data = train_df[train_df['Year'].isin(train_years)].copy()
val_data = train_df[train_df['Year'] == val_year].copy()

print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")


Splitting data into train and validation sets...
Training data shape: (74010, 17)
Validation data shape: (10950, 17)


In [565]:
train_data.head()

,ID,Year,Month,Day,kingdom,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
0,1,1,4,1,Arcadia,24.280002,-37.229980,25.50,30.50,8.5,10.3,22.52,58.89,16,8.6,283,1.648659
1,2,1,4,1,Atlantis,22.979999,-37.329990,299.65,305.15,5.9,8.2,22.73,11.83,12,15.8,161,1.583094
2,3,1,4,1,Avalon,22.880000,-37.130006,26.30,31.50,5.2,6.4,22.73,11.83,12,15.8,161,1.593309
3,4,1,4,1,Camelot,24.180003,-36.929994,24.00,28.40,8.2,10.7,22.67,75.27,16,6.4,346,1.638997
4,5,1,4,1,Dorne,25.780002,-37.530000,28.00,32.80,5.7,10.2,22.35,4.81,8,16.7,185,1.719189


In [568]:
val_data.head()

,ID,Year,Month,Day,kingdom,latitude,longitude,Avg_Temperature,Avg_Feels_Like_Temperature,Temperature_Range,Feels_Like_Temperature_Range,Radiation,Rain_Amount,Rain_Duration,Wind_Speed,Wind_Direction,Evapotranspiration
74010,74011,8,1,1,Arcadia,24.280002,-37.229980,25.40,29.70,7.1,8.2,18.33,4.29,6,5.1,129,1.477049
74011,74012,8,1,1,Atlantis,22.979999,-37.329990,300.55,305.95,6.6,8.8,21.34,3.25,6,9.4,8,1.621366
74012,74013,8,1,1,Avalon,22.880000,-37.130006,26.40,31.30,5.2,7.5,21.34,3.25,6,9.4,8,1.603420
74013,74014,8,1,1,Camelot,24.180003,-36.929994,22.60,26.20,4.8,6.1,16.90,3.25,10,7.2,117,1.368639
74014,74015,8,1,1,Dorne,25.780002,-37.530000,24.90,28.80,3.5,5.3,14.34,12.35,21,15.7,72,1.266948


In [571]:
def detect_and_handle_outliers(train_data, val_data, target_cols):
    """
    Detect and handle outliers in the target variables
    """
    print("\nDetecting and handling outliers...")

    # Create copies to avoid modifying originals
    train_clean = train_data.copy()
    val_clean = val_data.copy()

    # Handle outliers for each target variable by kingdom
    for kingdom in train_clean['kingdom'].unique():
        for col in target_cols:
            # Get kingdom specific data
            kingdom_data = train_clean[train_clean['kingdom'] == kingdom][col]

            # Calculate Q1, Q3 and IQR
            Q1 = kingdom_data.quantile(0.25)
            Q3 = kingdom_data.quantile(0.75)
            IQR = Q3 - Q1

            # Define bounds for outliers (1.5 IQR method)
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Replace outliers with bounds in training data
            train_clean.loc[(train_clean['kingdom'] == kingdom) &
                           (train_clean[col] < lower_bound), col] = lower_bound
            train_clean.loc[(train_clean['kingdom'] == kingdom) &
                           (train_clean[col] > upper_bound), col] = upper_bound

            # Apply same bounds to validation data
            val_clean.loc[(val_clean['kingdom'] == kingdom) &
                         (val_clean[col] < lower_bound), col] = lower_bound
            val_clean.loc[(val_clean['kingdom'] == kingdom) &
                         (val_clean[col] > upper_bound), col] = upper_bound
    return train_clean, val_clean

In [572]:
train_clean, val_clean = detect_and_handle_outliers(train_data, val_data, target_cols)


Detecting and handling outliers...


In [574]:
def preprocess_data(train_data, val_data, test_df, kingdom_temp_stats):
    """
    Preprocess data: convert temperature units, handle missing values
    """
    print("\nPreprocessing data...")

    # Create copies to avoid modifying originals
    train_processed = train_data.copy()
    val_processed = val_data.copy()
    test_processed = test_df.copy()

    # Get list of kingdoms using Kelvin
    kelvin_kingdoms = kingdom_temp_stats[kingdom_temp_stats['likely_unit'] == 'Kelvin'].index.tolist()
    print(f"Converting temperature for these kingdoms from Kelvin to Celsius: {kelvin_kingdoms}")

    # Convert Kelvin to Celsius in training data
    for kingdom in kelvin_kingdoms:
        kingdom_mask = train_processed['kingdom'] == kingdom
        train_processed.loc[kingdom_mask, 'Avg_Temperature'] = train_processed.loc[kingdom_mask, 'Avg_Temperature'] - 273.15
        train_processed.loc[kingdom_mask, 'Avg_Feels_Like_Temperature'] = train_processed.loc[kingdom_mask, 'Avg_Feels_Like_Temperature'] - 273.15

        # Also convert in validation data
        kingdom_mask = val_processed['kingdom'] == kingdom
        val_processed.loc[kingdom_mask, 'Avg_Temperature'] = val_processed.loc[kingdom_mask, 'Avg_Temperature'] - 273.15
        val_processed.loc[kingdom_mask, 'Avg_Feels_Like_Temperature'] = val_processed.loc[kingdom_mask, 'Avg_Feels_Like_Temperature'] - 273.15

    # Verify conversion worked
    print("\nAfter conversion, temperature statistics by kingdom:")
    combined_df = pd.concat([train_processed, val_processed])
    kingdom_temp_stats_after = combined_df.groupby('kingdom')['Avg_Temperature'].agg(['mean', 'min', 'max'])
    print(kingdom_temp_stats_after)

    # Check for extreme values after conversion
    print("\nChecking for extreme values after conversion:")
    for col in ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']:
        if col in train_processed.columns:
            print(f"{col}: Min={train_processed[col].min()}, Max={train_processed[col].max()}")

    return train_processed, val_processed, test_processed

In [575]:
train_processed, val_processed, test_processed = preprocess_data(train_clean, val_clean, test_df, kingdom_temp_stats)


Preprocessing data...
Converting temperature for these kingdoms from Kelvin to Celsius: ['Atlantis', 'El Dorado', 'Emerald City', 'Krypton', 'Nirvana', 'Olympus', 'Pandora', 'Rapture', 'Rivendell', 'Serenity', 'Solara', 'Utopia']

After conversion, temperature statistics by kingdom:
                   mean     min    max
kingdom                               
Arcadia       25.623446  22.800  28.40
Atlantis      27.212482  25.050  29.45
Avalon        26.599612  24.600  28.60
Camelot       23.587994  21.050  26.25
Dorne         27.552913  23.350  30.80
Eden          26.763242  24.300  29.10
El Dorado     25.214160  23.200  27.20
Elysium       26.663418  24.200  29.00
Emerald City  26.633704  24.450  28.85
Helios        25.872793  22.850  28.85
Krypton       26.807963  24.550  28.95
Metropolis    26.839689  24.400  29.20
Midgar        27.173799  24.400  30.00
Midgard       25.171716  21.600  28.80
Mordor        27.414566  24.050  30.10
Neo-City      26.669191  24.450  28.85
Neo-Tokyo    

In [576]:
def engineer_features(train_processed, val_processed, test_processed):
    """
    Create features: time-based, lag features, rolling windows, etc.
    """
    print("\nEngineering features...")

    # Create copies to avoid modifying originals
    train_featured = train_processed.copy()
    val_featured = val_processed.copy()
    test_featured = test_processed.copy()

    # Add time-based features
    def add_time_features(df):
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['day_of_year'] = df['date'].dt.dayofyear
        df['day_of_week'] = df['date'].dt.dayofweek
        df['quarter'] = df['date'].dt.quarter

        # Cyclical encoding for month, day of year, day of week
        df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
        df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

        df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
        df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)

        df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
        df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

        # Cyclical encoding for Wind_Direction if present
        if 'Wind_Direction' in df.columns:
            df['Wind_Direction_sin'] = np.sin(2 * np.pi * df['Wind_Direction']/360)
            df['Wind_Direction_cos'] = np.cos(2 * np.pi * df['Wind_Direction']/360)

        return df

    train_featured = add_time_features(train_featured)
    val_featured = add_time_features(val_featured)
    test_featured = add_time_features(test_featured)

    print("Added time-based features")

    # Define the target variables
    target_cols = ['Avg_Temperature', 'Radiation', 'Rain_Amount', 'Wind_Speed', 'Wind_Direction']

    # Merge train and validation for sequential feature creation
    # (Will split them back after feature engineering)
    train_val_merged = pd.concat([train_featured, val_featured]).sort_values(['kingdom', 'date'])

    # Create lag features and rolling window features by kingdom
    print("Creating lag features and rolling window features...")

    lag_periods = [1, 2, 3, 7, 14, 30]
    window_sizes = [7, 14, 30]

    for kingdom in train_val_merged['kingdom'].unique():
        kingdom_data = train_val_merged[train_val_merged['kingdom'] == kingdom]

        for col in target_cols:
            # Skip if column not present
            if col not in kingdom_data.columns:
                continue

            # Create lag features
            for lag in lag_periods:
                lag_col_name = f'{col}_lag_{lag}'
                train_val_merged.loc[train_val_merged['kingdom'] == kingdom, lag_col_name] = kingdom_data[col].shift(lag)

            # Create rolling window features
            for window in window_sizes:
                # Rolling mean
                window_mean_name = f'{col}_rolling_mean_{window}'
                train_val_merged.loc[train_val_merged['kingdom'] == kingdom, window_mean_name] = kingdom_data[col].rolling(window=window, min_periods=1).mean()

                # Rolling std
                window_std_name = f'{col}_rolling_std_{window}'
                train_val_merged.loc[train_val_merged['kingdom'] == kingdom, window_std_name] = kingdom_data[col].rolling(window=window, min_periods=1).std()

                # Rolling min/max
                window_min_name = f'{col}_rolling_min_{window}'
                train_val_merged.loc[train_val_merged['kingdom'] == kingdom, window_min_name] = kingdom_data[col].rolling(window=window, min_periods=1).min()

                window_max_name = f'{col}_rolling_max_{window}'
                train_val_merged.loc[train_val_merged['kingdom'] == kingdom, window_max_name] = kingdom_data[col].rolling(window=window, min_periods=1).max()

    print("Created lag and rolling window features")

    # Split back into train and validation
    train_featured = train_val_merged[train_val_merged['Year'].isin(list(range(1, 8)))].copy()
    val_featured = train_val_merged[train_val_merged['Year'] == 8].copy()

    # Create features for test data based on the latest values from training/validation
    print("Creating features for test data...")

    # For each kingdom and date in test data
    for kingdom in test_featured['kingdom'].unique():
        # Get the latest data from train/val for this kingdom
        latest_data = train_val_merged[train_val_merged['kingdom'] == kingdom].sort_values('date').iloc[-30:]

        # For each date in test for this kingdom
        test_kingdom = test_featured[test_featured['kingdom'] == kingdom].sort_values('date')

        for i, (idx, row) in enumerate(test_kingdom.iterrows()):
            # Create a temporary dataframe with the last 30 days of known data and this test row
            temp_df = pd.concat([latest_data, pd.DataFrame([row])]).reset_index(drop=True)

            # We need to create lag features for just this row
            # Since we don't have the actual target values for test data,
            # we'll need to create features based on what we have

            # For testing purposes, we'll just set some placeholder values
            # In a real implementation, you'd derive these from the latest training data
            for col in target_cols:
                for lag in lag_periods:
                    lag_col_name = f'{col}_lag_{lag}'
                    if lag <= len(latest_data):
                        test_featured.loc[idx, lag_col_name] = latest_data.iloc[-lag][col]
                    else:
                        test_featured.loc[idx, lag_col_name] = latest_data[col].mean()

                # Set rolling window features to the values from training data
                for window in window_sizes:
                    test_featured.loc[idx, f'{col}_rolling_mean_{window}'] = latest_data[col].mean()
                    test_featured.loc[idx, f'{col}_rolling_std_{window}'] = latest_data[col].std()
                    test_featured.loc[idx, f'{col}_rolling_min_{window}'] = latest_data[col].min()
                    test_featured.loc[idx, f'{col}_rolling_max_{window}'] = latest_data[col].max()

    # Handle missing values in features
    train_featured = train_featured.fillna(method='bfill').fillna(method='ffill')
    val_featured = val_featured.fillna(method='bfill').fillna(method='ffill')
    test_featured = test_featured.fillna(method='bfill').fillna(method='ffill')

    print(f"Final feature shapes - Train: {train_featured.shape}, Validation: {val_featured.shape}, Test: {test_featured.shape}")

    return train_featured, val_featured, test_featured

In [577]:
train_featured, val_featured, test_featured = engineer_features(train_processed, val_processed, test_processed)



Engineering features...


KeyError: 'date'